In [1]:
import sys
sys.path.append('../')
# from fragility.signalprocessing import frequencyanalysis
# from datainterface import readmat

# sys.path.append('/home/adamli/tng_tvb/')
# from tvbsim import visualize

import os
import time

import numpy as np
import pandas as pd
import scipy.io

import processing.frequencytransform as ft
import peakdetect
import processing.preprocessfft as preprocess
%load_ext autoreload
%autoreload 2

In [7]:
from scipy.signal import butter, lfilter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band', analog=False)
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = scipy.signal.filtfilt(b, a, data)
    return y
def butter_highpass(lowcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    b, a = butter(order, low, btype='highpass', analog=False)
    return b, a
def butter_highpass_filter(data, lowcut, fs, order=5):
    b, a = butter_highpass(lowcut, fs, order=order)
    y = scipy.signal.filtfilt(b, a, data)
    return y

def getseiztimes(onsettimes, offsettimes):
    minsize = np.min((len(onsettimes),len(offsettimes)))
    seizonsets = []
    seizoffsets = []
    
    # perform some checks
    if minsize == 0:
        print("no full onset/offset available!")
        return 0
    
    idx = 0
    # to store the ones we are checking rn
    _onset = onsettimes[idx]
    _offset = offsettimes[idx]
    seizonsets.append(_onset)
    
    # start loop after the first onset/offset pair
    for i in range(1,minsize):        
        # to store the previoius values
        _nextonset = onsettimes[i]
        _nextoffset = offsettimes[i]
        
        # check this range and add the offset if it was a full seizure
        # before the next seizure
        if _nextonset < _offset:
            _offset = _nextoffset
        else:
            seizoffsets.append(_offset)
            idx = i
            # to store the ones we are checking rn
            _onset = onsettimes[idx]
            _offset = offsettimes[idx]
            seizonsets.append(_onset)
    if len(seizonsets) != len(seizoffsets):
        seizonsets = seizonsets[0:len(seizoffsets)]
    return seizonsets, seizoffsets
def findonsetoffset(zts, delta=0.2/8):
    maxpeaks, minpeaks = peakdetect.peakdetect(zts, delta=delta)

    # get every other peaks
    onsettime, _ = zip(*minpeaks)
    offsettime, _ = zip(*maxpeaks)

    return onsettime, offsettime
def getonsetsoffsets(zts, ezindices, pzindices, delta=0.2/8):
    # create lambda function for checking the indices
    check = lambda indices: isinstance(indices,np.ndarray) and len(indices)>=1

    onsettimes=np.array([])
    offsettimes=np.array([])
    if check(ezindices):
        for ezindex in ezindices:
            _onsettimes, _offsettimes = findonsetoffset(zts[ezindex, :].squeeze(), 
                                                                    delta=delta)
            onsettimes = np.append(onsettimes, np.asarray(_onsettimes))
            offsettimes = np.append(offsettimes, np.asarray(_offsettimes))

    if check(pzindices):
        for pzindex in pzindices:
            _onsettimes, _offsettimes = findonsetoffset(zts[pzindex, :].squeeze(), 
                                                                    delta=delta)
            onsettimes = np.append(onsettimes, np.asarray(_onsettimes))
            offsettimes = np.append(offsettimes, np.asarray(_offsettimes))

    # first sort onsettimes and offsettimes
    onsettimes.sort()
    offsettimes.sort()
    
    return onsettimes, offsettimes

def computefreqindices(freqs, freqbands):
    freqbandindices = {}
    for band in freqbands:
        lowerband = freqbands[band][0]
        upperband = freqbands[band][1]

        # get indices where the freq bands are put in
        freqbandindices[band] = np.where((freqs >= lowerband) & (freqs < upperband))
        freqbandindices[band] = [freqbandindices[band][0][0], freqbandindices[band][0][-1]]
    return freqbandindices

def binFrequencyValues(power, freqs):
    # List of physiological frequency ranges
    freqbands = {
            'low':[0,14],
            'theta':[15,30],
            'gamma':[31,100],
            'high':[100,250]
    }
    
    # Create an empty array
    power_binned = np.zeros(shape=(power.shape[0],
                                    len(freqbands),
                                    power.shape[2]))
    
    # compute the freq indices for each band
    freqbandindices = computefreqindices(freqs,freqbands)

    # compress data using frequency bands
    for idx, band in enumerate(freqbandindices):
        indices = freqbandindices[band]
        # average between these two indices
        power_binned[:,idx,:] = np.mean(power[:,indices[0]:indices[1]+1,:], axis=1) #[np.newaxis,:,:]

    return power_binned

  
import ntpath
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)


# End of Transform code here

In [3]:
# FFT Parameters
fs = 500
winsize = 1000
stepsize = 500
typetransform = 'fourier'
mtbandwidth = 4
mtfreqs = []

mtaper = ft.MultiTaperFFT(winsize, stepsize, fs, mtbandwidth, mtfreqs)

Default method of tapering is eigen


In [12]:
datadir = '/Volumes/ADAM LI/pydata/tvbforwardsim/'
metadatadir = '/Volumes/ADAM LI/pydata/metadata/'    

# Get ALL datafiles from all downstream files
datafiles = []
for root, dirs, files in os.walk(datadir):
    for file in files:
        if file.endswith(".npz"):
             datafiles.append(os.path.join(root, file))
print(len(datafiles))
print(datafiles[50:])

70
['/Volumes/ADAM LI/pydata/tvbforwardsim/randmoved/_03/id002_cj/id002_cj_sim_nez1_npz0.npz', '/Volumes/ADAM LI/pydata/tvbforwardsim/randmoved/_03/id014_rb/id014_rb_sim_nez1_npz0.npz', '/Volumes/ADAM LI/pydata/tvbforwardsim/moved_v2/nez1_npz1/id001_ac_sim_nez1_npz1.npz', '/Volumes/ADAM LI/pydata/tvbforwardsim/moved_v2/nez1_npz1/id014_rb_sim_nez1_npz1.npz', '/Volumes/ADAM LI/pydata/tvbforwardsim/moved_v2/nez1_npz1/id002_cj_sim_nez1_npz1.npz', '/Volumes/ADAM LI/pydata/tvbforwardsim/moved_v2/nez1_npz3/id014_rb_sim_nez1_npz3.npz', '/Volumes/ADAM LI/pydata/tvbforwardsim/moved_v2/nez1_npz3/id001_ac_sim_nez1_npz3.npz', '/Volumes/ADAM LI/pydata/tvbforwardsim/moved_v2/nez1_npz3/id002_cj_sim_nez1_npz3.npz', '/Volumes/ADAM LI/pydata/tvbforwardsim/moved_v2/nez1_npz2/id014_rb_sim_nez1_npz2.npz', '/Volumes/ADAM LI/pydata/tvbforwardsim/moved_v2/nez1_npz2/id001_ac_sim_nez1_npz2.npz', '/Volumes/ADAM LI/pydata/tvbforwardsim/moved_v2/nez1_npz2/id002_cj_sim_nez1_npz2.npz', '/Volumes/ADAM LI/pydata/tvbfor

In [13]:
for idx, datafile in enumerate(datafiles[52:]):
    data = np.load(datafile, encoding='bytes')
    metadata = data['metadata'].item()
    zts = data['zts']
    seegts = data['seegts']
    
    # Extract location coordinates
    locations = metadata[b'seeg_xyz']
    try:
        patient_id = metadata[b'patient'].decode("utf-8") 
    except:
        patient = '_'.join(filename.split('_')[0:2])

    ezindices = metadata[b'ezindices']
    pzindices = metadata[b'pzindices']
    x0 = metadata[b'x0ez']
    seeg_contacts = metadata[b'seeg_contacts']
    
#     print(locations.shape)
#     print(zts.shape)
#     print(seegts.shape)
#     print(data.keys())
#     print(metadata.keys())
    # get onset/offset times correctly
    onsettimes, offsettimes = getonsetsoffsets(zts, np.array(ezindices), np.array(pzindices))

    # get the actual seizure times and offsets
    seizonsets, seizoffsets = getseiztimes(onsettimes, offsettimes)
    seizonsets = np.asarray(seizonsets)
    seizoffsets = np.asarray(seizoffsets)
    seiztimes = np.concatenate((seizonsets[:,np.newaxis], seizoffsets[:,np.newaxis]), axis=1)
    
#     print(seizonsets, seizoffsets)
#     print(seiztimes)
    # filter the data in case it was not done already
    lowcut = 0.1
    highcut = 499.
    fs = 1000.
    newseegts = butter_bandpass_filter(seegts, lowcut, highcut, fs, order=4)

    mtaper.loadrawdata(newseegts)
    power, freqs, timepoints, _ = mtaper.mtwelch()
    power = binFrequencyValues(power, freqs)
    
    filename = os.path.join('./traindata/', 
                            patient_id + '_nez' + str(len(ezindices)) + '_npz' + str(len(pzindices)) + '_'+str(idx) +'.npz')
    
    np.savez_compressed(filename, power=power, timepoints=timepoints, seiztimes=seiztimes,
                       locs=locations, seeg_contacts=seeg_contacts,x0ez=x0)
    print(power.shape)
    print(freqs.shape)
    print(timepoints.shape)
    print(locations.shape)
    print(seeg_contacts.shape)
    print('\n\n')

Loaded raw data in MultiTaperFFT!


/Users/adam2392/Documents/dnn-unsupervised/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:124: ComplexWarning: Casting complex values to real discards the imaginary part


(70, 4, 719)
(251,)
(719, 2)
(70, 3)
(70,)



Loaded raw data in MultiTaperFFT!
(165, 4, 719)
(251,)
(719, 2)
(165, 3)
(165,)



Loaded raw data in MultiTaperFFT!
(162, 4, 719)
(251,)
(719, 2)
(162, 3)
(162,)





ValueError: not enough values to unpack (expected 2, got 0)

In [10]:
# onsettimes, offsettimes = getonsetsoffsets(zts, np.array(ezindices), np.array(pzindices),delta=0.2)
print(datafile)

/Volumes/ADAM LI/pydata/tvbforwardsim/randmoved/_04/id002_cj/id002_cj_sim_nez1_npz0.npz


# Check what the transforms look like

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for i in range(powermultitaper.shape[0]):
    
#     time_series_limit = 200
    
    single_channel = np.real(powermultitaper[i])    

    plt.figure(figsize=(20,20))
    plt.imshow(single_channel, cmap='jet', interpolation='nearest')
    plt.show()
    
    single_channel_binned = binFrequencyValues(single_channel)
    
    plt.figure(figsize=(20,20))
    plt.imshow(single_channel_binned, cmap='jet', interpolation='nearest', aspect='auto')
    plt.show()
    

In [ ]:
for i in range(powermultitaper.shape[0]):
    
#     time_series_limit = 200
    
    single_channel = np.real(powermultitaper[i])    

    plt.figure(figsize=(20,20))
    plt.imshow(single_channel, cmap='jet', interpolation='nearest')
    plt.show()
    
    single_channel_binned = binFrequencyValues(single_channel)
    
    plt.figure(figsize=(20,20))
    plt.imshow(single_channel_binned, cmap='jet', interpolation='nearest', aspect='auto')
    plt.show()
    

In [ ]:
single_channel = np.real(powermultitaper[0])

x = binFrequencyValues(single_channel,frequency_ranges)

In [ ]:
x.shape

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(x[:,0:40], cmap='jet', interpolation='nearest')
plt.show()

In [ ]:
single_channel.shape

In [ ]:
# Extract frequency values from mtaper object
freqs = mtaper.freqsfft

In [ ]:
# Save with patient name
np.savez(patient_id + '.npz',powermultitaper,freqs,locations)

# numchannels * colours (binned frequencies, integral) * # of time windows...

# Write numpy array to file